# Libraries and input parameters

In [ ]:
# LIBRARIES #

import numpy as np

from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import LinearColorMapper, BasicTicker, ColorBar, Plot, CustomJS, ColumnDataSource, Rect
from bokeh.layouts import row, gridplot, column
from bokeh.models.widgets import Slider, Button
from bokeh.events import ButtonClick

from Gafchromic import Radiochromic_RB

from skimage import filters


output_notebook()

# Visualization interface

In [ ]:
# PLOTS IMAGE #
# @params:
#   img: array img to display
#   sizex: size of the img in x
#   sizey: size of the img in y
#   imgPlotWidth: width of the image plot

def simpleImgDisplay(img, sizex, sizey, imgPlotWidth):

    
    maxdose = int(np.amax(img))

    color_mapper = LinearColorMapper(palette="Viridis256", low=0, high=maxdose)

    color_bar = ColorBar(color_mapper=color_mapper, ticker=BasicTicker(),
                     label_standoff=12, border_line_color=None, location=(0,0),
                     title='Dose cGy')

    p1 = figure(plot_width=int(imgPlotWidth*1.1), plot_height=int(imgPlotWidth*sizey/sizex), 
                    x_range=(0,sizex), y_range=(0,sizey), 
                    title="Dose image", toolbar_location="above")


    p1.image(image=[img], x=[0], y=[0], dw=[sizex], dh=[sizey], color_mapper=color_mapper)

    p1.add_layout(color_bar, 'right')

    
    show(p1)

In [ ]:
# PLOTS IMAGE WITH ROIS#
# @params:
#   img: array img to display
#   sizex: size of the img in x
#   sizey: size of the img in y
#   imgPlotWidth: width of the image plot
#   rectList: list of rectangular ROIs to display
#   fillCol: Color to display the ROIs
#   alpha: color transparency of the ROIS

def simpleImgDisplayWithRois(img, sizex, sizey, imgPlotWidth, rectList, fillCol="#0096ff", alpha=0.2):

    
    # image display
    maxdose = int(np.amax(img))

    color_mapper = LinearColorMapper(palette="Viridis256", low=0, high=maxdose)

    color_bar = ColorBar(color_mapper=color_mapper, ticker=BasicTicker(),
                     label_standoff=12, border_line_color=None, location=(0,0),
                     title='Dose cGy')

    p1 = figure(plot_width=int(imgPlotWidth*1.1), plot_height=int(imgPlotWidth*sizey/sizex), 
                    x_range=(0,sizex), y_range=(0,sizey), 
                    title="Dose image", toolbar_location="above")


    p1.image(image=[img], x=[0], y=[0], dw=[sizex], dh=[sizey], color_mapper=color_mapper)

    p1.add_layout(color_bar, 'right')

    
    # Draws rectangles:
    for i in range(len(rectList)):
        p1.rect(x=int(rectList[i][0]+(rectList[i][2]-rectList[i][0])/2), 
           y=int(rectList[i][1]+(rectList[i][3]-rectList[i][1])/2), 
           width=rectList[i][2]-rectList[i][0],
           height=rectList[i][3]-rectList[i][1], 
           fill_color=fillCol, 
           alpha=alpha)

    
    
    # Organizing the graphs:
    show(p1)

In [ ]:
# PLOTS IMAGE AND PROFILES #
# @params:
#   img: array img to display
#   sizex: size of the img in x
#   sizey: size of the img in y
#   imgPlotWidth: width of the image plot

def imgDisplay(img, sizex, sizey, imgPlotWidth):


    # Img visualization:
    maxdose = int(np.amax(img))

    color_mapper = LinearColorMapper(palette="Viridis256", low=0, high=maxdose)

    color_bar = ColorBar(color_mapper=color_mapper, ticker=BasicTicker(),
                     label_standoff=12, border_line_color=None, location=(0,0),
                     title='Dose cGy')

    p1 = figure(plot_width=int(imgPlotWidth*1.1), plot_height=int(imgPlotWidth*sizey/sizex), 
                    x_range=(0,sizex), y_range=(0,sizey), 
                    title="Dose image", toolbar_location="above")

    l1_source = ColumnDataSource(data=dict(x=[0,int(sizex)], y=[int(sizey/2),int(sizey/2)]))

    p1.image(image=[img], x=[0], y=[0], dw=[sizex], dh=[sizey], color_mapper=color_mapper)
    p1.line('x', 'y', source=l1_source, line_width=2, line_color=(255, 255, 255, 0.7))

    p1.add_layout(color_bar, 'right')

    
    # Line profile:
    maxdose_x = np.amax(img[:,:])
    p2_source = ColumnDataSource(data=dict(x=np.arange(0, sizex, 1), y=img[int(sizey/2)]))
    p2 = figure(plot_width=imgPlotWidth, plot_height=300, x_range=(0,sizex), y_range=(0,maxdose_x), 
                    title="Horizontal dose profile", toolbar_location="above")
    p2.line('x', 'y', source=p2_source, line_color="#2690d4", line_width=3, line_alpha=1.0)

    
    # Callback functions called when the sliders are changed
    callback1 = CustomJS(args=dict(source1=l1_source, source2=p2_source), code="""
        var f = horizontalPosSlider.value;

        var data1 = source1.data;
        var y1 = data1['y'];
        y1[0] = f;
        y1[1] = f;
        source1.change.emit();

        var data2 = source2.data;
        var x2 = data2['x'];
        var y2 = data2['y'];
        var i;
        for (i = 0; i < x2.length; i++) {
            y2[i] = img[f][i];
        }    
        source2.change.emit();

    """)

    
    # slider:
    slider1 = Slider(start=0, end=sizey, value=int(sizey/2), step=1, 
                     title="Horizontal line position")
    slider1.js_on_change('value', callback1)
    callback1.args["horizontalPosSlider"] = slider1
    callback1.args["img"] = img

    
    # Organizing the graphs:
    grid = gridplot([[p1],[p2],[slider1]])
    show(grid)

# Main

In [ ]:
# INPUT PARAMETERS #

m_path = 'G:/Commun/PHYSICIENS/Erwann/EBT3/13 - etalonnage lot 02282001/scan 24h/'
#m_filename = 'etalonnage_24h_000'
m_filename = 'etalonnage_24h_0001.tif'
m_nbOfFiles = 1
m_firstNb = 1
m_splineFile = 'G:/Commun/PHYSICIENS/Erwann/EBT3/13 - etalonnage lot 02282001/scan 24h/bSpline_data rb.txt'

m_imgPlotWidth = 500
m_threshold = 45000


m_dosemax = 1200.0 #cGy

In [ ]:
g = Radiochromic_RB(m_path+m_filename, m_firstNb, m_nbOfFiles)
g.cropImg(800,2400,600,3800)
size = g.getSize()

#simpleImgDisplay(img1[:,:,0], size[0], size[1], 500)

In [ ]:
dispStatus = True

doseimg1 = g.convertToDose_cubicSplineFit_onlyFilm(m_splineFile, m_dosemax)

# attention si relancé une deuxième fois, _array a changé...
profiles, filteredProfiles, corr = g.correctScannerResponse(100,300, returnResults=dispStatus)

doseimg2 = g.convertToDose_cubicSplineFit_onlyFilm(m_splineFile, m_dosemax)



# Results display
if dispStatus:
    x = np.arange(0,len(profiles[0]))

    p1 = figure(plot_width=600, plot_height=400, title='Red profiles', toolbar_location="above")
    p1.line(x, profiles[0], line_width=2, line_color='firebrick', legend_label='original')
    p1.line(x, filteredProfiles[0], line_width=2, line_color='red', legend_label='filtered')

    p2 = figure(plot_width=600, plot_height=400, title='Green profiles', toolbar_location="above")
    p2.line(x, profiles[1], line_width=2, line_color='green', legend_label='original')
    p2.line(x, filteredProfiles[1], line_width=2, line_color='lightgreen', legend_label='filtered')
    
    p3 = figure(plot_width=600, plot_height=400, title='Blue profiles', toolbar_location="above")
    p3.line(x, profiles[2], line_width=2, line_color='blue', legend_label='original')
    p3.line(x, filteredProfiles[2], line_width=2, line_color='lightblue', legend_label='filtered')

    grid = gridplot([[p1],[p2],[p3]])
    show(grid)




In [ ]:
#simpleImgDisplay(doseimg1, size[0], size[1], 500)
#simpleImgDisplay(doseimg2, size[0], size[1], 500)

imgDisplay(doseimg1, size[0], size[1], 300)


In [ ]:
# CALCULATES THE DOSE IMAGE #

try:
    g = Radiochromic_RB(m_path+m_filename, m_firstNb, m_nbOfFiles)
    g.cropImg(800,2400,600,3800)
    
    rgbimg = g.getArray()
    doseimg = g.convertToDose_cubicSplineFit(m_splineFile, m_dosemax)
    size = g.getSize()
#    a = g.saveToTiff(doseimg, m_path+"converted_RB_Img.tif")
#    if a:
#        print("file written!")
except ValueError as err:
    print('Erreur: ' + err)
    



In [ ]:
rectList = []
rectList.append([0,100,1600,300])

#simpleImgDisplay(doseimg, size[0], size[1], m_imgPlotWidth)
#simpleImgDisplayWithRois(doseimg, size[0], size[1], m_imgPlotWidth, rectList, fillCol='#ede26e', alpha=0.3)
#imgDisplay(doseimg, size[0], size[1], m_imgPlotWidth)


In [ ]:
# Filters tests:
g = GafchromicFilms(m_path+m_filename, m_firstNb, m_nbOfFiles)
g.cropImg(800,2400,600,3800)
rgbimg = g.getArray()

r_img = rgbimg[:,:,0]
g_img = rgbimg[:,:,1]
b_img = rgbimg[:,:,2]

profileR = np.mean(r_img[100:300,0:1600], 0)
profileG = np.mean(g_img[100:300,0:1600], 0)
profileB = np.mean(b_img[100:300,0:1600], 0)


# Denoising:
filterValue = 25
filteredProfileR = filters.gaussian(profileR, filterValue)
filteredProfileG = filters.gaussian(profileG, filterValue)
filteredProfileB = filters.gaussian(profileB, filterValue)


# normalization and inversion (for a correction value):
mean = np.mean(filteredProfileR)
corrR = mean / filteredProfileR
mean = np.mean(filteredProfileG)
corrG = mean / filteredProfileG
mean = np.mean(filteredProfileB)
corrB = mean / filteredProfileB


# Display:

x = np.arange(0,len(profileR))
p1 = figure(plot_width=700, plot_height=400, title='RGB values', toolbar_location="above")

p1.line(x, profileR, line_width=2, line_color='firebrick')
p1.line(x, filteredProfileR, line_width=2, line_color='red')
#p1.line(x, corrR, line_width=2, line_color='firebrick')

show(p1)








